## **목표**
딥러닝 모델 학습 과정에서 발생하는 로그나 지표를 효과적으로 관리하고 시각화 하는 것은 모델의 성능을 개선하는데 중요한 역할을 한다.   
**Weights & Biases(wandb)**는 *학습 과정의 로그, 시각화, 하이퍼파라미터 튜닝 등의 기능을 제공*해 이런 지표들을 관리하고 시각화하는 과정을 간소하고 효율적으로 진행할 수 있게 한다.   
이번 실습을 통해 **wandb**의 기본 사용법과 PyTorch와의 연동 방법을 알아보자.

In [ ]:
!pip install wandb -q

### **1. 데이터 준비**

In [ ]:
import os
import wandb
import shutil
import urllib
import numpy as np
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from zipfile import ZipFile
from torchvision import datasets, models, transforms

%matplotlib inline

wandb.init(project = 'my-test-project', entity = '')

wandb: Currently logged in as: hcc9876 (nayoungpark). Use `wandb login --relogin` to force relogin


In [ ]:
# hymenoptera 데이터셋의 다운로드 링크
DATA_PATH = "https://download.pytorch.org/tutorial/hymenoptera_data.zip"

In [ ]:
# 위 링크에서 데이터셋을 다운로드하여 zip파일로 저장.
urllib.request.urlretrieve(DATA_PATH, "hymenoptera_data.zip")

# zip파일 압축 해제
with ZipFile("hymenoptera_data.zip", 'r') as zipObj:
   zipObj.extractall()

In [ ]:
data_dir = "./hymenoptera_data"

# 이미지 텐서의 형태 변경을 위한 사용자 정의 변환 클래스.
# 주어진 new_size로 이미지 텐서의 형태를 변경.
class ReshapeTransform:
    def __init__(self, new_size):
        self.new_size = new_size

    def __call__(self, img):
        result = torch.reshape(img, self.new_size)
        return result

# dataset 로드 시 사용한 변환들
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.ToTensor()
    ]),
    'val': transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.ToTensor()
    ]),
}

# 'train' 및 'val'폴더에 있는 이미지 데이터를 로드
# 각 이미지는 위에서 정의한 변환들을 거쳐서 로드 됨.
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}

# 학습 데이터셋 로드
# 전체 데이터셋을 한 번에 로드하기 때문에 배치 크기는 데이터셋의 전체 크기와 동일
train_dataset = torch.utils.data.DataLoader(image_datasets['train'],
                                            batch_size = len(image_datasets['train']),
                                            shuffle = True)

### **2. 모델 정의**

In [ ]:
# 사용자 정의 CNN 모델 클래스
class MyCNNModel(nn.Module):
    def __init__(self):
        super(MyCNNModel, self).__init__()

        # 첫 번째 합성곱 레이어
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, stride=2, padding=0),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))

        # 두 번째 합성곱 레이어
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=3, stride=2, padding=0),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))

        # 세 번째 합성곱 레이어
        self.layer3 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=0),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))

        # 드롭아웃과 완전 연결 레이어
        self.drop_out = nn.Dropout()
        self.fc1 = nn.Linear(3 * 3 * 64, 1000)
        self.fc2 = nn.Linear(1000, 1)

    # 순전파 함수 정의
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)

        out = out.view(out.size(0), -1)
        out = self.drop_out(out)
        out = self.fc1(out)
        out = self.fc2(out)
        return out

In [ ]:
EPOCHS = 25
BATCH_SIZE = 64
LEARNING_RATE = 0.01

In [ ]:
#모델 인스턴스 생성
model = MyCNNModel()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

criterion = nn. BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(),lr=LEARNING_RATE)

### **3. 학습 설정 및 실행**
wandb를 이용해서 학습 결과 로깅

In [ ]:
# 이진 분류 정확도 계산 함수
def binary_acc(y_pred, y_test):
    y_pred_tag = torch.round(torch.sigmoid(y_pred))
    correct_results_sum = (y_pred_tag == y_test).sum().float()
    acc = correct_results_sum/y_test.shape[0]
    acc = torch.round(acc * 100)

    return acc

In [ ]:
# wandb 설정
config={'epochs': EPOCHS, 'batch_size':BATCH_SIZE, 'learning_rate':LEARNING_RATE}
wandb.init(project = 'my-test-project', config=config)

In [ ]:
wandb.init(project="my-test-project", config=config)
# wandb.config.batch_size = BATCH_SIZE
# wandb.config.learning_rate = LEARNING_RATE
# config={"epochs": EPOCHS, "batch_size": BATCH_SIZE, "learning_rate" : LEARNING_RATE}

# 학습 루프
for e in range(1, EPOCHS+1):
    epoch_loss = 0
    epoch_acc = 0
    for X_batch, y_batch in train_dataset:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device).type(torch.cuda.FloatTensor)
        optimizer.zero_grad()
        y_pred = model(X_batch)

        loss = criterion(y_pred, y_batch.unsqueeze(1))
        acc = binary_acc(y_pred, y_batch.unsqueeze(1))

        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        epoch_acc += acc.item()

    train_loss = epoch_loss/len(train_dataset)
    train_acc = epoch_acc/len(train_dataset)
    print(f'Epoch {e+0:03}: | Loss: {train_loss:.5f} | Acc: {train_acc:.3f}')
    wandb.log({'accuracy': train_acc, 'loss': train_loss})

Epoch 001: | Loss: 0.70804 | Acc: 53.000
Epoch 002: | Loss: 31.89152 | Acc: 50.000
Epoch 003: | Loss: 1.08492 | Acc: 52.000
Epoch 004: | Loss: 7.16965 | Acc: 50.000
Epoch 005: | Loss: 1.96751 | Acc: 51.000
Epoch 006: | Loss: 5.02726 | Acc: 50.000
Epoch 007: | Loss: 5.25379 | Acc: 50.000
Epoch 008: | Loss: 2.53696 | Acc: 50.000
Epoch 009: | Loss: 0.94235 | Acc: 47.000
Epoch 010: | Loss: 2.07238 | Acc: 50.000
Epoch 011: | Loss: 1.95571 | Acc: 50.000
Epoch 012: | Loss: 0.95475 | Acc: 51.000
Epoch 013: | Loss: 1.44487 | Acc: 50.000
Epoch 014: | Loss: 2.28253 | Acc: 50.000
Epoch 015: | Loss: 1.66169 | Acc: 51.000
Epoch 016: | Loss: 0.71069 | Acc: 53.000
Epoch 017: | Loss: 1.12074 | Acc: 58.000
Epoch 018: | Loss: 1.39205 | Acc: 56.000
Epoch 019: | Loss: 1.15874 | Acc: 58.000
Epoch 020: | Loss: 0.79217 | Acc: 60.000
Epoch 021: | Loss: 0.68976 | Acc: 61.000
Epoch 022: | Loss: 0.83121 | Acc: 58.000
Epoch 023: | Loss: 0.96848 | Acc: 57.000
Epoch 024: | Loss: 0.81725 | Acc: 59.000
Epoch 025: | Lo